# [모듈 6.1] 모델 배포 파이프라인 개발 (SageMaker Model Building Pipeline 모든 스텝)

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 **약 5분** 소요 됩니다.

- 0. SageMaker Model Building Pipeline 개요
- 1. 파이프라인 변수 및 환경 설정
- 2. 파이프라인 스텝 단계 정의

    - (1) 모델 승인 상태 변경 람다 스텝    
    - (2) 배포할 세이지 메이커 모델 스텝 생성
    - (3) 모델 앤드 포인트 배포를 위한 람다 스텝 생성    
    
- 3. 모델 빌딩 파이프라인 정의 및 실행
- 4. Pipleline 캐싱 및 파라미터 이용한 실행
- 5. 정리 작업
    
---

# 0.[모듈 6.1] 모델 배포 파이프라인 개요

- 이 노트북은 다음과 같은 상황에 대한 파이프라인 입니다.
    - 모델 레제스트리에 여러개의 모델 패키지 그룹이 있습니다.
    - 모델 패키지 그룹에서 특정한 것을 선택하여 가장 최근에 저장된 모델 버전을 선택 합니다.
    - 선택된 모델 버전의 "모델 승인 상태"를 "Pending" 에서 "Approved" 로 변경 합니다.
    - 이 모델 버전에 대해서 세이지 메이커 모델을 생성합니다.
    - 세이지 메이커 모델을 기반으로 앤드포인트를 생성 합니다.

# 1. 파이프라인 변수 및 환경 설정



In [1]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

sm_client = boto3.client('sagemaker', region_name=region)

%store -r 

## (1) 모델 빌딩 파이프라인 변수 생성

파이프라인에 인자로 넘길 변수는 아래 크게 3가지 종류가 있습니다.
- 모델 레지스트리에 모델 등록시에 모델 승인 상태 값    


In [2]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)


model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)


# 2. 파이프라인 스텝 단계 정의

## (1) 모델 승인 상태 변경 람다 스텝
- 모델 레지스트리에서 해당 모델 패키지 그룹을 조회하고, 가장 최신 버전의 모델에 대해서 '모델 승인 상태 변경' 을 합니다.


#### [에러] 
아래와 같은 데러가 발생시에 `0.0.Setup-Environment.ipynb` 의 정책 추가 부분을 진행 해주세요.
```
ClientError: An error occurred (AccessDenied) when calling the CreateRole operation: User: arn:aws:sts::0287032915XX:assumed-role/AmazonSageMaker-ExecutionRole-20210827T141955/SageMaker is not authorized to perform: iam:CreateRole on resource: arn:aws:iam::0287032915XX:role/lambda-deployment-role
```

In [3]:
from src.iam_helper import create_lambda_role

lambda_role = create_lambda_role("lambda-deployment-role")
print("lambda_role: \n", lambda_role)

Using ARN from existing role: lambda-deployment-role
lambda_role: 
 arn:aws:iam::028703291518:role/lambda-deployment-role


In [4]:
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)

import time 

current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())
function_name = "sagemaker-lambda-step-approve-model-deployment-" + current_time

print("function_name: \n", function_name)

function_name: 
 sagemaker-lambda-step-approve-model-deployment-08-27-12-08-42


In [5]:
# Lambda helper class can be used to create the Lambda function
func_approve_model = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script="src/iam_change_model_approval.py",
    handler="iam_change_model_approval.lambda_handler",
)

output_param_1 = LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String)
output_param_2 = LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)
output_param_3 = LambdaOutput(output_name="other_key", output_type=LambdaOutputTypeEnum.String)

step_approve_lambda = LambdaStep(
    name="LambdaApproveModelStep",
    lambda_func=func_approve_model,
    inputs={
        "model_package_group_name" : model_package_group_name,
        "ModelApprovalStatus": "Approved",
    },
    outputs=[output_param_1, output_param_2, output_param_3],
)


## (2) 배포할 세이지 메이커 모델 스텝 생성
- 위의 람다 스텝에서 "모델 승인 상태" 를 변경한 모델에 대하여 '모델 레지스트리'에서 저장된 도커 컨테이너 이미지, 모델 아티펙트의 위치를 가져 옵니다.
- 이후에 이 두개의 인자를 가지고 세이지 메이커 모델을 생성 합니다.

In [6]:
import boto3
sm_client = boto3.client('sagemaker')

# 위에서 생성한 model_package_group_name 을 인자로 제공 합니다.
response = sm_client.list_model_packages(ModelPackageGroupName= model_package_group_name)

ModelPackageArn = response['ModelPackageSummaryList'][0]['ModelPackageArn']
sm_client.describe_model_package(ModelPackageName=ModelPackageArn)
response = sm_client.describe_model_package(ModelPackageName=ModelPackageArn)
image_uri_approved = response["InferenceSpecification"]["Containers"][0]["Image"]
ModelDataUrl_approved = response["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]
print("image_uri_approved: ", image_uri_approved)
print("ModelDataUrl_approved: ", ModelDataUrl_approved)

image_uri_approved:  683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3
ModelDataUrl_approved:  s3://sagemaker-us-east-1-028703291518/fraud2train/training_jobs/pipelines-ebt5t8rln4ye-FraudTrain-c21jj7fu99/output/model.tar.gz


In [7]:
from sagemaker.model import Model
    
model = Model(
    image_uri= image_uri_approved,
    model_data= ModelDataUrl_approved,    
    sagemaker_session=sagemaker_session,
    role=role,
)

In [8]:
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.steps import CreateModelStep


inputs = CreateModelInput(
    instance_type="ml.m5.large",
    # accelerator_type="ml.eia1.medium",
)
step_create_best_model = CreateModelStep(
    name="CreateFraudhModel",
    model=model,
    inputs=inputs,
)
step_create_best_model.add_depends_on([step_approve_lambda]) # step_approve_lambda 완료 후 실행 함.

## (3) 모델 앤드 포인트 배포를 위한 람다 스텝 생성
- 람다 함수는 입력으로 세이지 메이커 모델, 앤드 포인트 컨피그 및 앤드 포인트 이름을 받아서, 앤드포인트를 생성 함.


In [9]:
# model_name = project_prefix + "-lambda-model" + current_time
endpoint_config_name = "lambda-deploy-endpoint-config-" + current_time
endpoint_name = "lambda-deploy-endpoint-" + current_time

function_name = "sagemaker-lambda-step-endpoint-deploy-" + current_time

# print("model_name: \n", model_name)
print("endpoint_config_name: \n", endpoint_config_name)
print("endpoint_config_name: \n", len(endpoint_config_name))
print("endpoint_name: \n", endpoint_name)
print("function_name: \n", function_name)




endpoint_config_name: 
 lambda-deploy-endpoint-config-08-27-12-08-42
endpoint_config_name: 
 44
endpoint_name: 
 lambda-deploy-endpoint-08-27-12-08-42
function_name: 
 sagemaker-lambda-step-endpoint-deploy-08-27-12-08-42


In [10]:
# Lambda helper class can be used to create the Lambda function
func_deploy_model = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script="src/iam_create_endpoint.py",
    handler="iam_create_endpoint.lambda_handler",
    timeout = 900, # 디폴트는 120초 임. 10분으로 연장
)

output_param_1 = LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String)
output_param_2 = LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)
output_param_3 = LambdaOutput(output_name="other_key", output_type=LambdaOutputTypeEnum.String)

step_deploy_lambda = LambdaStep(
    name="LambdaDeployStep",
    lambda_func=func_deploy_model,
    inputs={
        "model_name": step_create_best_model.properties.ModelName,
        "endpoint_config_name": endpoint_config_name,
        "endpoint_name": endpoint_name,
    },
    outputs=[output_param_1, output_param_2, output_param_3],
)

# 3.모델 빌딩 파이프라인 정의 및 실행
위에서 정의한 아래의 4개의 스텝으로 파이프라인 정의를 합니다.
-     steps=[step_process, step_train, step_create_model, step_deploy],
- 아래는 약 1분 정도 소요 됩니다.  이후 아래와 같이 실행 결과를 스튜디오에서 확인할 수 있습니다.


- ![deployment-pipeline.png](img/deployment-pipeline.png)

In [11]:
from sagemaker.workflow.pipeline import Pipeline

project_prefix = 'sagemaker-pipeline-phase2-deployment-step-by-step'

pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        model_approval_status,        
    ],

    
  steps=[step_approve_lambda, step_create_best_model, step_deploy_lambda],
)



In [13]:
import json

definition = json.loads(pipeline.definition())
# definition

### 파이프라인을 SageMaker에 제출하고 실행하기 


In [14]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:028703291518:pipeline/sagemaker-pipeline-phase2-deployment-step-by-step',
 'ResponseMetadata': {'RequestId': '798d1f51-864e-4984-b17b-88e61d6332b0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '798d1f51-864e-4984-b17b-88e61d6332b0',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '117',
   'date': 'Fri, 27 Aug 2021 12:08:43 GMT'},
  'RetryAttempts': 0}}

디폴트값을 이용하여 파이프라인을 샐행합니다. 

In [15]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

실행이 완료될 때까지 기다립니다.

In [16]:
execution.wait()

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

In [17]:
execution.list_steps()

[{'StepName': 'LambdaDeployStep',
  'StartTime': datetime.datetime(2021, 8, 27, 12, 8, 50, 90000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 8, 27, 12, 8, 52, 510000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'Lambda': {'Arn': 'arn:aws:lambda:us-east-1:028703291518:function:sagemaker-lambda-step-endpoint-deploy-08-27-12-08-42',
    'OutputParameters': [{'Name': 'other_key', 'Value': 'example_value'},
     {'Name': 'body', 'Value': '"Created Endpoint!"'},
     {'Name': 'statusCode', 'Value': '200.0'}]}}},
 {'StepName': 'CreateFraudhModel',
  'StartTime': datetime.datetime(2021, 8, 27, 12, 8, 49, 130000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 8, 27, 12, 8, 49, 941000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:028703291518:model/pipelines-u9fpxxg4051d-createfraudhmodel-0ucxigidgh'}}},
 {'StepName': 'LambdaApproveModelStep',
  'StartTime': datetime.datetime(2021, 8, 27, 12,

# 5. 정리 작업


## 변수 저장

In [21]:
depolyment_endpoint_name = endpoint_name
%store depolyment_endpoint_name

all_deployment_pipeline_name = pipeline_name
%store all_deployment_pipeline_name

Stored 'depolyment_endpoint_name' (str)
Stored 'all_deployment_pipeline_name' (str)
